### Module

In [18]:
import os
import sys
from dotenv import load_dotenv
import textwrap
from IPython.display import display
from IPython.display import Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


### SQL DB에 크롤링 페이지 분류

In [ ]:
import mysql.connector
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# MySQL 연결 설정
mysql_config = {
    'user': 'root',
    'password': 'admin',
    'host': '175.45.205.178',
    'port': '3306',
    'database': 'vectorstoreDB'
}

# Create a connection to the MySQL database
try:
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    print("Successfully connected to the database")

    # Your database operations here

except mysql.connector.Error as err:
    print(f"Error: {err}")

# finally:
#     if connection.is_connected():
#         cursor.close()
#         connection.close()
#         print("MySQL connection is closed")


In [ ]:
# 테이블 형태에 맞게 불러온다
#아래는 불러오는 SQL DB의 테이블 형태이다.
"""
{"id":"000","url":"https:링크...",
"content":"국민기초생활지원 Text:....신청은..누가...",
"title":"국민기초생활지원"}
"""

In [ ]:


# 각 문서의 벡터와 메타데이터를 MySQL에 저장
for doc in results['documents']:
    document_id = doc['id']
    vector = doc['embedding']
    metadata = doc['metadata']

    # 벡터를 BLOB 형식으로 변환
    vector_blob = vector.tobytes()

    # 데이터베이스에 삽입
    cursor.execute(
        "INSERT INTO embeddings (document_id, vector, metadata) VALUES (%s, %s, %s)",
        (document_id, vector_blob, json.dumps(metadata))
    )

# 변경사항 커밋
connection.commit()

# 연결 종료
cursor.close()
connection.close()


코드 작성 플로우 sql에 있는 텍스트 더미를 하나씩 받는다
-> sql에 있던 텍스트를 프롬프팅한 모델에 넣고 하나씩 분류


In [ ]:
db_format_info="""
title : 제목
date : 날짜
category1 : 카테고리1
category2 : 카테고리2
details : 상세내용

"""
db_format="""   
    {
      'title': '자녀 학업 장려금 지원 (행사 정보 요약)',
      'date': '2024. 7. 1.(월) ~ 26.(금)',
      'category1': '교육',
      'category2': '장학금',
      'details': '자녀 학업 장려금에 대한 상세 내용이 여기에 들어갑니다.\n'
          '자녀 학업 장려금에 대한 상세 내용이 여기에 들어갑니다.'
    },
    {
      'title': '청소년 여름캠프 안내',
      'date': '2024. 8. 10.(토) ~ 15.(목)',
      'category1': '캠프',
      'category2': '청소년',
      'details': '청소년 여름캠프에 대한 상세 내용이 여기에 들어갑니다.',
    },
    {
      'title': 'test1',
      'date': '2024. 7. 1.(월) ~ 26.(금)',
      'category1': '11',
      'category2': '111',
      'details': 'test111111',
    },
    {
      'title': 'test2',
      'date': '2024. 8. 10.(토) ~ 15.(목)',
      'category1': '22',
      'category2': '222',
      'details': 'test22222',
    }
    """

### dummy test(강남대 학적 정보 답변 DB)

In [ ]:
import os
import sys
from dotenv import load_dotenv
import textwrap
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        return super().__call__(input)

embeddings = CustomOpenAIEmbeddings(api_key=OPENAI_API_KEY)


In [ ]:
def load_pdf_files(pdf_directory):
    documents = []
    print("Start to load PDFs.")
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        print(f"Loaded {len(pdf_documents)} documents from {pdf_file}")
        documents.extend(pdf_documents)
    print(f"Loaded {len(documents)} PDF documents.")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_db(chunks, embeddings, vector_db_path):
    if not chunks:
        print("No chunks to save to VectorDB.")
        return None
    print("Save VectorDB")
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=vector_db_path)
    vectordb.add_documents(documents=chunks)
    vectordb.persist()
    return vectordb.as_retriever()

def setup_category_vector_db(category):
    pdf_directory = f"/home/user/khtml-ai-llm-tt/data/academic resources/{category}"
    vector_db_path = f"/home/user/khtml-ai-llm/vectordb/{category}"

    documents = load_pdf_files(pdf_directory)
    chunks = split_documents(documents)
    retriever = save_db(chunks, embeddings, vector_db_path)

    return retriever


#### category 별 vectordb 생성

In [ ]:
categories = ['다전공', '수강', '전공', '학적']
retrievers = {category: setup_category_vector_db(category) for category in categories}

print("모든 카테고리에 대한 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")


In [ ]:
print(retrievers)

#### Model

In [ ]:
import json
SYS_PROMPT = """
다음은 강남대학교 AI 로드맵을 위한 참고 자료입니다:
{context}
사용자 입력 메시지에 잘 따라주세요:
{question}
"""

template = SYS_PROMPT + '''
1. 강남대학교 AI 로드맵을 위한 참고 자료야: {context}
2. 사용자 입력 메시지에 잘 따라줘야 해: {question}
'''

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model='gpt-4o', temperature=0)
rag_chains={}
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)
# Suggested code may be subject to a license. Learn more: ~LicenseLog:3204574595.
for retriever in retrievers.values():
    rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
#     rag_chains["catego"]    rag_chains[category] = (
#     {'context': retriever | format_docs, 'question': RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )



def get_response(category, user_input):
    retriever = retrievers[category]
    rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
    input_message = f"카테고리: {category}\n사용자 입력: {user_input}"
    return rag_chain.invoke(input_message)

# 카테고리 "복지"와 각 키워드에 대해 답변을 생성하여 JSON 형식으로 저장
responses = []
for keyword in categories:
    user_input = "자퇴와 계절수업에 대해서 각각 알려줘"  # 예시 입력
    response_text = get_response(keyword, user_input)
    response_json = {
        "category": "복지",
        "keyword": keyword,
        "messages": response_text
    }
    responses.append(response_json)
    print(response_text)

# JSON 파일로 저장
output_file = "responses.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(responses, f, ensure_ascii=False, indent=4)

print(f"Responses have been saved to {output_file}.")

### app test(복지 정보 DB)

In [ ]:
import os
import sys
from dotenv import load_dotenv
import textwrap
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

class CustomOpenAIEmbeddings(OpenAIEmbeddings):
    def __call__(self, input):
        return super().__call__(input)

embeddings = CustomOpenAIEmbeddings(api_key=OPENAI_API_KEY)


In [ ]:
def load_pdf_files(pdf_directory):
    documents = []
    print("Start to load PDFs.")
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    print(f"Found {len(pdf_files)} PDF files.")
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        print(f"Loaded {len(pdf_documents)} documents from {pdf_file}")
        documents.extend(pdf_documents)
    print(f"Loaded {len(documents)} PDF documents.")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks

def save_db(chunks, embeddings, vector_db_path):
    if not chunks:
        print("No chunks to save to VectorDB.")
        return None
    print("Save VectorDB")
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=vector_db_path)
    vectordb.add_documents(documents=chunks)
    vectordb.persist()
    return vectordb.as_retriever()

def setup_category_vector_db(category):
    pdf_directory = f"/home/user/khtml-ai-llm-tt/data/welfare/{category}"
    vector_db_path = f"/home/user/khtml-ai-llm/vectordb/{category}"

    documents = load_pdf_files(pdf_directory)
    chunks = split_documents(documents)
    retriever = save_db(chunks, embeddings, vector_db_path)

    return retriever


#### category 별 vectordb 생성

In [ ]:
categories = ['저소득 복지', '보훈대상자 지원', '사회복지시설', '어르신', '장애인', '아동', '여성', '청년', '주거복지']
retrievers = {category: setup_category_vector_db(category) for category in categories}

print("모든 카테고리에 대한 PDF 파일이 벡터 데이터베이스에 저장되었습니다.")


## Classes

#### Variables

In [57]:
SQLDB_directory=""

#DB 이름 정해지면 수정
#카테고리-키워드 모음
keyword = ['저소득복지', '보훈대상자지원', '사회복지시설', '어르신', '장애인', '아동', '여성', '청년', '주거복지']


text_dummy="""
신청은 누가, 어떻게 하나요? 생활이 어려운 저
소득 가구의 가구원 및 그 친족, 기타 이해관계인이 신청 거주지 동 주민센터에 직접 방문하여 상담 및 신청(연중 수시)가능 대상자 선정기준은 어떻게 되나요? 가구별로 조사·산정된 소득인정액이 기준중위소득을 기준으로 하여 생계,의료,주거,교육 급여별 선정기준 이하일 경우 수급자로 선정될 수 있습니다. 2024년도 가구별 기준중위소득의 가구규모, 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년 가구별 기준중위소득 가구규모 1인 2인 3인 4인 5인 6인 7인 2024(원/월) 2,228,445 3,682,609 4,714,657 5,729,913 6,695,735 7,618,369 8,514,994 2024년도 급여별 선정기준의 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년도 급여별 선정기준 1인 2인 3인 4인 5인 6인 7인 생계급여(32%)기준 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 의료급여(40%)기준 891,378 1,473,044 1,885,863 2,291,965 2,678,294 3,047,348 3,405,998 주거급여(48%) 기준 1,069,654 1,767,652 2,263,035 2,750,358 3,213,953 3,656,817 4,087,197 교육급여(50%) 기준 1,114,222 1,841,305 2,357,328 2,864,956 3,347,867 3,809,184 4,257,497 ※ 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정 부양의무자 기준 적용되는 수급자 종류 부양의무자 기준 적용 : 의료급여 부양의무자 기준 미적용 : 생계급여＊, 주거급여, 교육급여 ＊단, 부양의무자의 연 소득 1억원(월소득 834만원) 또는 일반재산 9억원을 초과하는 경우 생계급여 대상에서 제외 소득인정액은 어떻게 산정되나요? 소득평가액과 재산의 소득환산액을 합한 것으로 소득평가액에는 실제소득, 부양의무자의 부양비, 사적이전소득 등이 포함되며, 재산의 소득환산액은 기본재산액과 부채를 공제한 후 일반재산과 금융재산의 소득환산율을 적용한 금액을 말합니다. ※ 기본재산액(공제대상 재산가액) 기본재산액(공제대상 재산가액) 구분 서울 경기(용인) 광역·세종·창원 그 외 지역 생계·의료·주거·교육급여 9,900만원 8,000만원 7,700만원 5,300만원 ※ 소득환산율 소득환산율 가구수 주거용재산 일반재산 금융재산 수급권자 월 1.04% 월 4.17% 월 6.26% 부양의무자 월 1.04% 월 2.08% 급여의 종류와 지원방법은? 급여의 종류에는 생계급여, 의료급여, 주거급여, 교육급여, 해산·장제급여가 있으며, 매월 20일 정기적으로 지급되는 생계·주거급여는 현금급여기준에서 조사된 가구의 소득인정액을 차감하여 산정한 금액을 해당가구의 계좌에 입금하는 것을 원칙으로 하고 있습니다. 해산·장제급여는 출생증명서나 사망진단서를 첨부하여 신청하시면 됩니다. 의료급여는 의료기관(병·의원, 약국) 이용시 본인부담금을 감면받을 수 있는 의료급여증이 발급되며, 현금으로 지급되는 것은 아닙니다. 2024년 생계급여 현금급여 지급기준의 가구수, 1인, 2인, 3인, 4인, 5인, 6인,7인의 정보전달 2024년 생계급여 현금급여 지급 기준 가구수 1인 2인 3인 4인 5인 6인 7인 생계급여 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 8인 이상 가구의 선정 및 급여기준 : 7인가구 생계급여 선정기준액에서 6인가구 생계급여 선정기준액을 차감한 금액인 286,920원을 추가함 (8인가구 생계급여 선정기준액 3,011,718원) 2023년 주거급여 임차가구 기준임대료의 가구수, 1인, 2인, 3인, 4인, 5인, 6인,7인의 정보전달 2024년 주거급여 임차가구 기준임대료 가구수 1인 2인 3인 4인 5인 6~7인 기준임대료 (2급지) 268,000 300,000 358,000 414,000 428,000 507,000 ※ 임차료가 아닌 별도대가를 지불하는 사용대차 관계 등에 대하여는 실제 임차료가 기준임대료의 60%인 것으로 간주하여 지급합니다. 수급자의 신고의무란? 수급자로 선정되어 보호받는 자는 해당 가구의 급여종류별 수급자격 및 금액, 관리주체 등의 결정에 영향을 끼치는 모든 변동사항에 대하여 신고의 의무가 있음(국민기초생활보장법 제37조) 이를 위반하여 부정수급이 확인되었을 시에는 그간 지급한 급여의 전부 또는 일부를 징수할 수 있으며, 이와 별도로 1년 이하의 징역, 1천만원 이하의 벌금, 구류 또는 과료에 처할 수 있음(법 제49조)"}이거 정리해줘
"""
# 데이터 포맷 정보
db_format_info="""
주어진 데이터를 아래의 형식으로 변환해주세요. 각 항목에 맞는 정보를 추출하고, 반드시 JSON 형식으로 출력하세요.

### 매우 중요한 사항:
1. 절대로 `\n` 문자가 포함되지 않도록 모든 줄바꿈 문자를 완벽하게 제거하고, 모든 텍스트를 한 줄로 작성해야 합니다. 줄바꿈 문자가 남아있어서는 절대 안 됩니다.
2. 필요한 정보는 제공된 텍스트에서 정확하게 추출하여 채워넣습니다.
3. 누락된 정보가 있을 경우 해당 항목은 빈 문자열로 비워둡니다.
4. 각 항목의 설명은 반드시 한국어로 작성되어야 합니다.
5. "keyword"의 원소는 최대 3개까지로 제한하며, 반드시 다음 리스트에서 선택해야 합니다:
   ['저소득복지', '보훈대상자지원', '사회복지시설', '어르신', '장애인', '아동', '여성', '청년', '주거복지'].

### 변환할 데이터 형식 (JSON):
{
  "img_url": "",
  "title": "지원대상자 신청 안내",
  "category": "복지",
  "keyword": "",
  "eligibility": "생활이 어려운 저소득 가구",
  "start_date": "",
  "end_date": "",
  "contact": "거주지 동 주민센터",
  "content": "생활이 어려운 저소득 가구의 가구원 및 그 친족, 기타 이해관계인이 신청할 수 있으며, 거주지 동 주민센터에 직접 방문하여 상담 및 신청 가능합니다. 대상자 선정기준은 가구별로 조사·산정된 소득인정액이 기준중위소득 이하일 경우 수급자로 선정될 수 있습니다.",
  "applyMethod": "거주지 동 주민센터 직접 방문"
}

### 주어진 데이터:
{text_dummy}
"""

db_format_info_eng="""
Convert the given data into the format below, extracting the relevant information for each field, and absolutely ensure the output is in JSON format.

### Extremely Important:
1. **Absolutely NO** `\n` (newline) characters are allowed in the output. All newline characters **MUST** be completely removed. Every piece of text **MUST** be written on a single line. If any newline characters are present, the output is considered invalid.
2. Extract the required information accurately from the provided text and populate each field accordingly.
3. If any information is missing, leave the respective field as an empty string.
4. All descriptions for each field **MUST** be written in Korean.
5. The "keyword" field **MUST** contain no more than 1 element, and this element **MUST** be selected only from the following list:
   ['저소득복지', '보훈대상자지원', '사회복지시설', '어르신', '장애인', '아동', '여성', '청년', '주거복지'].

### JSON Format to Convert to:
{
  "img_url": "",
  "title": "",
  "category": "",
  "keyword": "",
  "eligibility": "",
  "start_date": "",
  "end_date": "",
  "contact": "",
  "content": "",
  "applyMethod": ""
}

### Given Data:
{text_dummy}
"""


### DBManager

In [63]:
import mysql.connector
import json

class DatabaseManager:
    def __init__(self, config):
        self.config = config
        self.connection = None
        self.cursor = None
    
    def connect(self):
        # Connect to the database
        try:
            self.connection = mysql.connector.connect(**self.config)
            print(1)
            self.cursor = self.connection.cursor()
            print("Successfully connected to the database")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
    
    def close(self):
        # Close the cursor and connection
        if self.connection.is_connected():
            self.cursor.close()
            self.connection.close()
            print("MySQL connection is closed")
    
    def fetch_crawled_data(self):
        # Fetch crawled data from the database
        query = "SELECT id, url, content, title FROM crawled_data"
        self.cursor.execute(query)
        return self.cursor.fetchall()
    SQLDB_directory=""
    #DB 이름 정해지면 수정
    def insert_processed_data(self, document_id, vector, metadata):
        vector_blob = vector.tobytes()
        query = "INSERT INTO {SQLDB_directory} (document_id, vector, metadata) VALUES (%s, %s, %s)"
        self.cursor.execute(query, (document_id, vector_blob, json.dumps(metadata)))
        self.connection.commit()


In [72]:
    mysql_config = {
        'user': 'root',
        'password': 'admin',
        'host': '175.45.205.178',
        'port': '3306',
        'database': 'khtml'
    }
    
    # 데이터베이스 매니저 초기화 및 연결
    db_manager = DatabaseManager(mysql_config)
    db_manager.connect()
    
    # 크롤링된 데이터 가져오기
    crawled_data = db_manager.fetch_crawled_data()

1
Successfully connected to the database


In [73]:
crawled_data

[(1,
  'https://www.yongin.go.kr/home/www/www01/www01_01/www01_01_02/www01_01_02_01.jsp',
  '신청은 누가, 어떻게 하나요? 생활이 어려운 저소득 가구의 가구원 및 그 친족, 기타 이해관계인이 신청 거주지 동 주민센터에 직접 방문하여 상담 및 신청(연중 수시)가능 대상자 선정기준은 어떻게 되나요? 가구별로 조사·산정된 소득인정액이 기준중위소득을 기준으로 하여 생계,의료,주거,교육 급여별 선정기준 이하일 경우 수급자로 선정될 수 있습니다. 2024년도 가구별 기준중위소득의 가구규모, 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년 가구별 기준중위소득 가구규모 1인 2인 3인 4인 5인 6인 7인 2024(원/월) 2,228,445 3,682,609 4,714,657 5,729,913 6,695,735 7,618,369 8,514,994 2024년도 급여별 선정기준의 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년도 급여별 선정기준 1인 2인 3인 4인 5인 6인 7인 생계급여(32%)기준 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 의료급여(40%)기준 891,378 1,473,044 1,885,863 2,291,965 2,678,294 3,047,348 3,405,998 주거급여(48%) 기준 1,069,654 1,767,652 2,263,035 2,750,358 3,213,953 3,656,817 4,087,197 교육급여(50%) 기준 1,114,222 1,841,305 2,357,328 2,864,956 3,347,867 3,809,184 4,257,497 ※ 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정 부양의무자 기준 적용되는 수급자 종류 부양의무자 기준 적용 : 의료급여 부양의무자 기준 미적용 

### DBProcessor


크롤링된 데이터를 sqlDB로 올리기 위해 json format으로 변경해준다.

#### codedsdaf

In [87]:
import time
from datetime import datetime, timedelta

class DataProcessor:
    def __init__(self, model, db_format, db_format_info):
        self.model = model
        self.db_format_info = db_format_info
        
        # 시스템 프롬프트 정의
        SYS_PROMPT = '''
        JSON 형태로 답변이 나와야해
        {{key:value, key:value, key:value.... }} 이 형태로
        다음 기준에 맞춰서 요약해줘 : {db_format_info}
        '''
        
        # 프롬프트 템플릿 설정
        template = SYS_PROMPT + '''
        사용자 입력 메세지에 잘 따라줘야해 : {question}
        '''
        
        # 프롬프트 설정
        self.prompt = ChatPromptTemplate.from_template(template)
        
        # 체인 구성
        self.chain = (
            {'db_format_info': RunnablePassthrough(), 'question': RunnablePassthrough()}
            | self.prompt
            | self.model
            | StrOutputParser()
        )

    def tuple_to_dict(self, entry):
        return {
            'id': entry[0],
            'url': entry[1],
            'content': entry[2],
            'category': entry[3]
        }

    def process_data(self, data):
        start_time = time.time()
        total_entries = len(data)
        results = []

        for index, entry in enumerate(data):
            entry = self.tuple_to_dict(entry)
            processed_entry = self._process_single_entry(entry)
            results.append(self._clean_result(processed_entry))
            
            # 진행 상황 출력
            elapsed_time = time.time() - start_time
            avg_time_per_entry = elapsed_time / (index + 1)
            estimated_time_remaining = avg_time_per_entry * (total_entries - (index + 1))
            progress_percent = (index + 1) / total_entries * 100

            # 남은 시간 계산
            remaining_time = timedelta(seconds=int(estimated_time_remaining))
            print(f"진행 횟수: {index + 1}/{total_entries} ({progress_percent:.2f}%)")
            print(f"남은 시간: {remaining_time}")

        return results
    
    def _process_single_entry(self, entry):
        # 데이터와 프롬프트를 체인에 전달하여 결과를 받음
        result = self.chain.invoke({'db_format_info': self.db_format_info, 'question': entry['content']})
        return result
    
    def _clean_result(self, result):
        # \n 문자를 공백으로 치환
        cleaned_result = result.replace('\n', ' ')
        return cleaned_result

# ChatOpenAI 인스턴스 생성
model = ChatOpenAI(model='gpt-4o', temperature=0, api_key=OPENAI_API_KEY)

# DataProcessor 인스턴스에 모델과 데이터 포맷 정보 전달
dp = DataProcessor(model=model, db_format=db_format, db_format_info=db_format_info_eng)

# 입력 데이터 예시
text_dummy = [
    {
        'id': 1,
        'content': """
        신청은 누가, 어떻게 하나요? 생활이 어려운 저
        소득 가구의 가구원 및 그 친족, 기타 이해관계인이 신청 거주지 동 주민센터에 직접 방문하여 상담 및 신청(연중 수시)가능 대상자 선정기준은 어떻게 되나요? 가구별로 조사·산정된 소득인정액이 기준중위소득을 기준으로 하여 생계,의료,주거,교육 급여별 선정기준 이하일 경우 수급자로 선정될 수 있습니다. 2024년도 가구별 기준중위소득의 가구규모, 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년 가구별 기준중위소득 가구규모 1인 2인 3인 4인 5인 6인 7인 2024(원/월) 2,228,445 3,682,609 4,714,657 5,729,913 6,695,735 7,618,369 8,514,994 2024년도 급여별 선정기준의 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년도 급여별 선정기준 1인 2인 3인 4인 5인 6인 7인 생계급여(32%)기준 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 의료급여(40%)기준 891,378 1,473,044 1,885,863 2,291,965 2,678,294 3,047,348 3,405,998 주거급여(48%) 기준 1,069,654 1,767,652 2,263,035 2,750,358 3,213,953 3,656,817 4,087,197 교육급여(50%) 기준 1,114,222 1,841,305 2,357,328 2,864,956 3,347,867 3,809,184 4,257,497 ※ 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정 부양의무자 기준 적용되는 수급자 종류 부양의무자 기준 적용 : 의료급여 부양의무자 기준 미적용 : 생계
        """
    },
    # ... (추가 데이터 항목)
]

# 데이터 처리
processing_results = dp.process_data(crawled_data)
print(processing_results)


진행 횟수: 1/79 (1.27%)
남은 시간: 0:17:52
진행 횟수: 2/79 (2.53%)
남은 시간: 0:13:11
진행 횟수: 3/79 (3.80%)
남은 시간: 0:11:32
진행 횟수: 4/79 (5.06%)
남은 시간: 0:12:06
진행 횟수: 5/79 (6.33%)
남은 시간: 0:10:15
진행 횟수: 6/79 (7.59%)
남은 시간: 0:09:18
진행 횟수: 7/79 (8.86%)
남은 시간: 0:08:56
진행 횟수: 8/79 (10.13%)
남은 시간: 0:08:17
진행 횟수: 9/79 (11.39%)
남은 시간: 0:08:17
진행 횟수: 10/79 (12.66%)
남은 시간: 0:07:53
진행 횟수: 11/79 (13.92%)
남은 시간: 0:09:11


KeyboardInterrupt: 

In [53]:

for processing_result in processing_results:
    #print(to_markdown(processing_result))
    print(processing_result)

{   "img_url": "",   "title": "저소득 가구 지원",   "category": "복지",   "category-keywords": "저소득복지",   "eligibility": "생활이 어려운 저소득 가구의 가구원 및 그 친족, 기타 이해관계인",   "start_date": "",   "end_date": "",   "contact": "",   "content": "가구별로 조사·산정된 소득인정액이 기준중위소득을 기준으로 하여 생계, 의료, 주거, 교육 급여별 선정기준 이하일 경우 수급자로 선정될 수 있습니다. 2024년도 가구별 기준중위소득은 다음과 같습니다: 1인 가구 2,228,445원, 2인 가구 3,682,609원, 3인 가구 4,714,657원, 4인 가구 5,729,913원, 5인 가구 6,695,735원, 6인 가구 7,618,369원, 7인 가구 8,514,994원. 급여별 선정기준은 생계급여(32%) 1인 713,102원, 2인 1,178,435원, 3인 1,508,690원, 4인 1,833,572원, 5인 2,142,635원, 6인 2,437,878원, 7인 2,724,798원. 의료급여(40%) 1인 891,378원, 2인 1,473,044원, 3인 1,885,863원, 4인 2,291,965원, 5인 2,678,294원, 6인 3,047,348원, 7인 3,405,998원. 주거급여(48%) 1인 1,069,654원, 2인 1,767,652원, 3인 2,263,035원, 4인 2,750,358원, 5인 3,213,953원, 6인 3,656,817원, 7인 4,087,197원. 교육급여(50%) 1인 1,114,222원, 2인 1,841,305원, 3인 2,357,328원, 4인 2,864,956원, 5인 3,347,867원, 6인 3,809,184원, 7인 4,257,497원. 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정합니다. 부양의무자 기

### VectorStoreManager

DB_Processor에서 만든 JSON FORMAT을 chunking하여 임베딩하고 벡터스토어에 저장한다

In [54]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

class VectorStoreManager:
    def __init__(self, persist_directory,embeddings):
        self.persist_directory = persist_directory
        self.store = Chroma(persist_directory=persist_directory)
        self.embeddings=embeddings
   
        #chromadb를 이용한 로컬 저장 구현
        
    # def save_vectors(self, data):
    #     for entry in data:
    #         self.store.add(entry['id'], entry['embedding'], entry['metadata'])
    #     self.store.persist()
    def
    def split_documents(documents, chunk_size=1000, chunk_overlap=200):
        chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        chunks = chunk_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")
    return chunks
    
    def save_db(chunks, vector_db_path):
        if not chunks:
            print("No chunks to save to VectorDB.")
            return None
        print("Save VectorDB")
        vectordb = Chroma.from_documents(documents=chunks, embedding=self.embeddings, persist_directory=vector_db_path)
        #vectordb.add_documents(documents=chunks)
        vectordb.persist()
        return vectordb.as_retriever()

    def vector_store_process(self, documents):
        chunks = self.split_documents(documents)
        retriever = self.save_db(chunks, self.persist_directory)
        return retriever



SyntaxError: invalid syntax (3077450953.py, line 10)

In [ ]:

vsm=VectorStoreManager(persist_directory="/home/user/khtml-ai-llm-tt/vectordb",embeddings=embeddings)
vsm.vector_store_process(processing_results)

### main

In [5]:
def main():
    # MySQL 연결 설정
    mysql_config = {
        'user': 'root',
        'password': 'admin',
        'host': '175.45.205.178',
        'port': '3306',
        'database': 'vectorstoreDB'
    }
    
    # 데이터베이스 매니저 초기화 및 연결
    db_manager = DatabaseManager(mysql_config)
    db_manager.connect()
    
    # 크롤링된 데이터 가져오기
    crawled_data = db_manager.fetch_crawled_data()
    
    # 데이터 처리기 초기화
    gpt_model = OpenAIEmbeddings()  # GPT-4 모델로 교체 필요
    data_processor = DataProcessor(gpt_model)
    
    # 데이터 처리
    processed_data = data_processor.process_data(crawled_data)
    
    # 벡터 저장소 매니저 초기화 및 벡터 저장
    vector_store_manager = VectorStoreManager('/path/to/persist_directory')
    vector_store_manager.save_vectors(processed_data)
    
    # 처리된 데이터 SQL DB에 저장
    for data in processed_data:
        db_manager.insert_processed_data(data['id'], data['embedding'], data['metadata'])
    
    # 연결 종료
    db_manager.close()

if __name__ == "__main__":
    main()


### Test

In [2]:
text_dummy="""
신청은 누가, 어떻게 하나요? 생활이 어려운 저
소득 가구의 가구원 및 그 친족, 기타 이해관계인이 신청 거주지 동 주민센터에 직접 방문하여 상담 및 신청(연중 수시)가능 대상자 선정기준은 어떻게 되나요? 가구별로 조사·산정된 소득인정액이 기준중위소득을 기준으로 하여 생계,의료,주거,교육 급여별 선정기준 이하일 경우 수급자로 선정될 수 있습니다. 2024년도 가구별 기준중위소득의 가구규모, 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년 가구별 기준중위소득 가구규모 1인 2인 3인 4인 5인 6인 7인 2024(원/월) 2,228,445 3,682,609 4,714,657 5,729,913 6,695,735 7,618,369 8,514,994 2024년도 급여별 선정기준의 1인, 2인, 3인, 4인, 5인, 6인, 7인의 정보전달 2024년도 급여별 선정기준 1인 2인 3인 4인 5인 6인 7인 생계급여(32%)기준 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 의료급여(40%)기준 891,378 1,473,044 1,885,863 2,291,965 2,678,294 3,047,348 3,405,998 주거급여(48%) 기준 1,069,654 1,767,652 2,263,035 2,750,358 3,213,953 3,656,817 4,087,197 교육급여(50%) 기준 1,114,222 1,841,305 2,357,328 2,864,956 3,347,867 3,809,184 4,257,497 ※ 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정 부양의무자 기준 적용되는 수급자 종류 부양의무자 기준 적용 : 의료급여 부양의무자 기준 미적용 : 생계급여＊, 주거급여, 교육급여 ＊단, 부양의무자의 연 소득 1억원(월소득 834만원) 또는 일반재산 9억원을 초과하는 경우 생계급여 대상에서 제외 소득인정액은 어떻게 산정되나요? 소득평가액과 재산의 소득환산액을 합한 것으로 소득평가액에는 실제소득, 부양의무자의 부양비, 사적이전소득 등이 포함되며, 재산의 소득환산액은 기본재산액과 부채를 공제한 후 일반재산과 금융재산의 소득환산율을 적용한 금액을 말합니다. ※ 기본재산액(공제대상 재산가액) 기본재산액(공제대상 재산가액) 구분 서울 경기(용인) 광역·세종·창원 그 외 지역 생계·의료·주거·교육급여 9,900만원 8,000만원 7,700만원 5,300만원 ※ 소득환산율 소득환산율 가구수 주거용재산 일반재산 금융재산 수급권자 월 1.04% 월 4.17% 월 6.26% 부양의무자 월 1.04% 월 2.08% 급여의 종류와 지원방법은? 급여의 종류에는 생계급여, 의료급여, 주거급여, 교육급여, 해산·장제급여가 있으며, 매월 20일 정기적으로 지급되는 생계·주거급여는 현금급여기준에서 조사된 가구의 소득인정액을 차감하여 산정한 금액을 해당가구의 계좌에 입금하는 것을 원칙으로 하고 있습니다. 해산·장제급여는 출생증명서나 사망진단서를 첨부하여 신청하시면 됩니다. 의료급여는 의료기관(병·의원, 약국) 이용시 본인부담금을 감면받을 수 있는 의료급여증이 발급되며, 현금으로 지급되는 것은 아닙니다. 2024년 생계급여 현금급여 지급기준의 가구수, 1인, 2인, 3인, 4인, 5인, 6인,7인의 정보전달 2024년 생계급여 현금급여 지급 기준 가구수 1인 2인 3인 4인 5인 6인 7인 생계급여 713,102 1,178,435 1,508,690 1,833,572 2,142,635 2,437,878 2,724,798 8인 이상 가구의 선정 및 급여기준 : 7인가구 생계급여 선정기준액에서 6인가구 생계급여 선정기준액을 차감한 금액인 286,920원을 추가함 (8인가구 생계급여 선정기준액 3,011,718원) 2023년 주거급여 임차가구 기준임대료의 가구수, 1인, 2인, 3인, 4인, 5인, 6인,7인의 정보전달 2024년 주거급여 임차가구 기준임대료 가구수 1인 2인 3인 4인 5인 6~7인 기준임대료 (2급지) 268,000 300,000 358,000 414,000 428,000 507,000 ※ 임차료가 아닌 별도대가를 지불하는 사용대차 관계 등에 대하여는 실제 임차료가 기준임대료의 60%인 것으로 간주하여 지급합니다. 수급자의 신고의무란? 수급자로 선정되어 보호받는 자는 해당 가구의 급여종류별 수급자격 및 금액, 관리주체 등의 결정에 영향을 끼치는 모든 변동사항에 대하여 신고의 의무가 있음(국민기초생활보장법 제37조) 이를 위반하여 부정수급이 확인되었을 시에는 그간 지급한 급여의 전부 또는 일부를 징수할 수 있으며, 이와 별도로 1년 이하의 징역, 1천만원 이하의 벌금, 구류 또는 과료에 처할 수 있음(법 제49조)"}이거 정리해줘
"""

db_format_info="""
title : 제목
date : 날짜(없으면 NULL 처리)
category1 : 카테고리1
category2 : 카테고리2
details : 상세내용

category1 : 복지
category2 : [ 1. 저소득 복지,2. 보훈대상자 지원,3. 사회복지시설,4. 어르신,5. 장애인,6. 아동,7. 여성,8. 청년,9. 주거복지]
1. 저소득 복지
2. 보훈대상자 지원
3. 사회복지시설
4. 어르신
5. 장애인
6. 아동
7. 여성
8. 청년
9. 주거복지
"""

In [3]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트 정의
SYS_PROMPT = '''
다음 기준에 맞춰서 요약해줘 : {db_format_info}
'''

# 프롬프트 템플릿 설정
template = SYS_PROMPT + '''
사용자 입력 메세지에 잘 따라줘야해 : {question}
'''

# 프롬프트 설정
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0, api_key=OPENAI_API_KEY)

# 체인 구성
chain = (
    {'db_format_info': RunnablePassthrough(), 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


# 입력 데이터와 db_format_info를 체인에 전달하여 결과를 받습니다.
result = chain.invoke({'db_format_info': db_format_info, 'question': text_dummy})

# 결과 출력
print(result)


```json
{
  "title": "저소득 가구 지원 신청 및 선정 기준",
  "date": "NULL",
  "category1": "복지",
  "category2": "저소득 복지",
  "details": "신청은 생활이 어려운 저소득 가구의 가구원, 친족, 기타 이해관계인이 거주지 동 주민센터에 직접 방문하여 상담 및 신청 가능(연중 수시). 대상자 선정기준은 가구별 소득인정액이 기준중위소득을 기준으로 생계, 의료, 주거, 교육 급여별 선정기준 이하일 경우 수급자로 선정. 2024년도 가구별 기준중위소득은 1인 가구 2,228,445원, 2인 가구 3,682,609원, 3인 가구 4,714,657원, 4인 가구 5,729,913원, 5인 가구 6,695,735원, 6인 가구 7,618,369원, 7인 가구 8,514,994원. 급여별 선정기준은 생계급여(32%) 1인 713,102원, 2인 1,178,435원, 3인 1,508,690원, 4인 1,833,572원, 5인 2,142,635원, 6인 2,437,878원, 7인 2,724,798원. 의료급여(40%) 1인 891,378원, 2인 1,473,044원, 3인 1,885,863원, 4인 2,291,965원, 5인 2,678,294원, 6인 3,047,348원, 7인 3,405,998원. 주거급여(48%) 1인 1,069,654원, 2인 1,767,652원, 3인 2,263,035원, 4인 2,750,358원, 5인 3,213,953원, 6인 3,656,817원, 7인 4,087,197원. 교육급여(50%) 1인 1,114,222원, 2인 1,841,305원, 3인 2,357,328원, 4인 2,864,956원, 5인 3,347,867원, 6인 3,809,184원, 7인 4,257,497원. 8인 이상 가구는 7인가구 기준 중위소득에서 6인가구 기준 중위소득의 차액을 7인가구 기준 중위소득에 더하여 산정. 부양의무자 기준 적용되는 수급자 종류는 의료급여, 부양의무자 기준 미적용은 생계."
}
